# Deploy cohere-multilingual-small Model Package from AWS Marketplace 


Cohere builds a collection of Large Language Models (LLMs) trained on a massive corpus of curated web data. Powering these models, our infrastructure enables our product to be deployed for a wide range of use cases. The use cases we power include generation (copy writing, etc), summarization, classification, content moderation, information extraction, semantic search, and contextual entity extraction

This sample notebook shows you how to deploy [cohere-multilingual-small](TODO: https://aws.amazon.com/marketplace/pp/prodview-6dmzzso5vu5my) using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

## Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to [cohere-multilingual-small](TODO: https://aws.amazon.com/marketplace/pp/prodview-6dmzzso5vu5my). If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

## Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.-Visualize-output)
   5. [Writing a blobpost with co.generate](#E.-writing-a-blobpost-with-cogenerate)
   6. [Entity Extraction using co.generate](#F.-entity-extraction-using-cogenerate)
   7. [Article Summarization using co.generate](#G.-article-summarization-using-cogenerate)
   5. [Delete the endpoint](#H.-Delete-the-endpoint)
3. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page [cohere-multilingual-small](TODO: https://aws.amazon.com/marketplace/pp/prodview-6dmzzso5vu5my)
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [ ]:
# Mapping for Model Packages (initially only us-east-1 and eu-west-1 is supported)
model_package_map = {
    "us-east-1": "arn:aws:sagemaker:us-east-1:865070037744:model-package/cohere-multilinguat-small-v1-0-3f54af2dd44e33b9bd378ea637417bdf",
    "eu-west-1": "arn:aws:sagemaker:eu-west-1:985815980388:model-package/cohere-multilinguat-small-v1-0-3f54af2dd44e33b9bd378ea637417bdf",
}

: 

In [ ]:
!pip install cohere-sagemaker

from cohere_sagemaker import Client, CohereError

In [ ]:
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
import numpy as np

In [ ]:
region = boto3.Session().region_name
if region not in model_package_map.keys():
    raise ("UNSUPPORTED REGION")

model_package_arn = model_package_map[region]

In [ ]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket = sagemaker_session.default_bucket()
runtime = boto3.client("runtime.sagemaker")
bucket

In [ ]:
co = Client(endpoint_name='cohere-multilingual-small')

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [ ]:
model_name = "cohere-multilingual-small"

content_type = "application/json"

real_time_inference_instance_type = (
    "ml.g5.xlarge"
)

### A. Create an endpoint

In [ ]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

Once endpoint has been created, you would be able to perform real-time inference.

### B. Create input payload

In [ ]:
texts = [
    "When are you open?", 
    "When do you close?", 
    "What are the hours?", 
    "Are you open on weekends?", 
    "Are you available on holidays?", 
    "How much is a burger?", 
    "What\'s the price of a meal?", 
    "How much for a few burgers?", 
    "Do you have a vegan option?", 
    "Do you have vegetarian?", 
    "Do you serve non-meat alternatives?", 
    "Do you have milkshakes?", 
    "Milkshake", 
    "Do you have desert?", 
    "Can I bring my child?", 
    "Are you kid friendly?", 
    "Do you have booster seats?", 
    "Do you do delivery?", 
    "Is there takeout?", 
    "Do you deliver?", 
    "Can I have it delivered?", 
    "Can you bring it to me?", 
    "Do you have space for a party?", 
    "Can you accommodate large groups?", 
    "Can I book a party here?"
]

### C. Perform real-time inference

In [ ]:
response = co.embed(texts=texts, truncate='none')

### D. Visualize output

In [ ]:
print(f'Embeddings: {response.embeddings}') 

### H. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Clean-up

### A. Delete the model

In [ ]:
model.delete_model()

### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

